In [1]:
import pandas as pd

roc_test = pd.read_csv("data/roc_nlu_test.csv")

In [2]:
roc_test.head()

,Unnamed: 0,context,head,relation,tail,label
0,0,['hank had to wrap a lot of gifts for his fami...,PersonX wraps gifts,xWant,give gifts,1
1,1,['hank had to wrap a lot of gifts for his fami...,PersonX wraps gifts,xWant,to deliver the gifts,0
2,2,['hank had to wrap a lot of gifts for his fami...,gift,AtLocation,wrapped container,1
3,3,['hank had to wrap a lot of gifts for his fami...,gift,AtLocation,gift shop,0
4,4,['hank had to wrap a lot of gifts for his fami...,gift,ObjectUse,bring for the birthday host,0


In [41]:
roc_test_ones = roc_test[roc_test["label"] == 1]
roc_test_zeros = roc_test[roc_test["label"] == 0]

In [42]:
roc_samples = pd.concat([roc_test_ones.sample(50), roc_test_zeros.sample(50)])

In [43]:
roc_samples.to_csv("data/roc_samples.csv", index=False)

In [1]:
import pandas as pd

roc_samples = pd.read_csv("data/roc_samples.csv")

In [2]:
roc_samples.head()

,Unnamed: 0,context,head,relation,tail,label
0,5125,"['faith was learning to play the piano .', 'sh...",practice,ObjectUse,perfect skill,1
1,5818,['kyle asked a girl to go to the movies with h...,PersonX invites PersonY to a movie,xIntent,to see a movie with a friend.,1
2,1408,['ben wanted to learn how to practice ballet ....,PersonX wants to be a ballerina,xNeed,to ger enrolled into a dance school,1
3,2978,['lawrence was visiting summer camp for the fi...,PersonX loves one 's ___,xEffect,BE FRIEND,1
4,4912,"[""jack 's friend works as an insurance broker ...",PersonX gives PersonY a job,xNeed,hire PersonY,1


In [3]:
from kogito.inference import CommonsenseInference

csi = CommonsenseInference()

In [4]:
csi.processors

{'head': ['sentence_extractor',
  'noun_phrase_extractor',
  'verb_phrase_extractor'],
 'relation': ['simple_relation_matcher', 'graph_relation_matcher']}

In [5]:
from kogito.models.bart.comet import COMETBART

model = COMETBART.from_pretrained("mismayil/comet-bart-ai2")

In [6]:
from kogito.linkers.deberta import DebertaLinker

linker = DebertaLinker()

In [7]:
def text_to_list(text):
    return [t.strip().strip("'") for t in text.strip("]").strip("[").split(",")]

In [15]:
from tqdm import tqdm

roc_results = []
roc_subsamples = roc_samples.sample(50)

for index, row in tqdm(roc_subsamples.iterrows(), total=len(roc_subsamples), leave=True):
    output_graph = csi.infer(text=row["head"], model=model, model_args={"num_generate": 1})
    context = text_to_list(row["context"])
    probs = linker.link(context=context, input_graph=output_graph)

    for output_kg, tail_probs in zip(output_graph, probs):
        roc_results.append({
            "text": row["head"],
            "context": row["context"],
            "head": output_kg.head,
            "relation": output_kg.relation, 
            "tail": output_kg.tails[0], 
            "prediction": 1 if tail_probs[0] >= 0.5 else 0
        })

  0%|          | 0/50 [00:00<?, ?it/s]

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


  2%|▏         | 1/50 [00:14<11:59, 14.69s/it]

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


  4%|▍         | 2/50 [00:22<08:16, 10.35s/it]

...

 96%|█████████▌| 48/50 [10:39<00:28, 14.28s/it]

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


 98%|█████████▊| 49/50 [11:00<00:16, 16.49s/it]

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


100%|██████████| 50/50 [11:17<00:00, 13.55s/it]


In [19]:
roc_full_pipeline_results = []

for res in roc_results:
    roc_full_pipeline_results.append({
        "text": res["text"],
        "context": res["context"],
        "head": str(res["head"]),
        "relation": str(res["relation"]),
        "tail": res["tail"],
        "prediction": res["prediction"]
    })

In [22]:
import random

roc_full_pipeline_samples = random.sample(roc_full_pipeline_results, 100)

In [23]:
import json

with open("data/roc_full_pipeline_result_samples.json", "w") as f:
    json.dump(roc_full_pipeline_samples, f, indent=2)

In [21]:
import json

with open("data/roc_full_pipeline_results.json", "w") as f:
    json.dump(roc_full_pipeline_results, f, indent=2)

In [24]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import json

with open("data/roc_full_pipeline_result_samples.json") as f:
    roc_results = json.load(f)

labels = [roc_result["label"] for roc_result in roc_results]
predictions = [roc_result["prediction"] for roc_result in roc_results]

print(f"Accuracy={accuracy_score(labels, predictions)}, F1={f1_score(labels, predictions)}, Recall={recall_score(labels, predictions)}, Precision={precision_score(labels, predictions)}")

Accuracy=0.78, F1=0.7441860465116279, Recall=0.9142857142857143, Precision=0.6274509803921569


In [44]:
from kogito.core.knowledge import KnowledgeGraph

input_graph = KnowledgeGraph.from_csv("data/roc_samples.csv", tails_col="tail")

In [45]:
input_graph[:2]

[Knowledge(head="practice", relation="ObjectUse", tails=['perfect skill']),
 Knowledge(head="PersonX invites PersonY to a movie", relation="xIntent", tails=['to see a movie with a friend.'])]

In [20]:
from kogito.models.bart.comet import COMETBART

model = COMETBART.from_pretrained("mismayil/comet-bart-ai2")

In [46]:
output_graph = model.generate(input_graph, num_generate=1)

100%|██████████| 2/2 [00:08<00:00,  4.39s/it]


In [47]:
output_graph[:2]

[Knowledge(head="practice", relation="ObjectUse", tails=[' practice']),
 Knowledge(head="PersonX invites PersonY to a movie", relation="xIntent", tails=[' to have fun'])]

In [48]:
roc_results = []

for roc_sample, output_sample in zip(roc_samples.itertuples(), output_graph):
    roc_results.append({
        "context": roc_sample.context,
        "head": roc_sample.head,
        "relation": roc_sample.relation,
        "reference_tails": roc_sample.tail,
        "generated_tails": output_sample.tails,
        "reference_label": roc_sample.label
    })

In [49]:
import json

with open("data/roc_results.json", "w") as f:
    json.dump(roc_results, f, indent=2)

In [34]:
from kogito.linkers.deberta import DebertaLinker

linker = DebertaLinker()

In [35]:
def text_to_list(text):
    return [t.strip().strip("'") for t in text.strip("]").strip("[").split(",")]

In [36]:
text_to_list(roc_test.iloc[0]["context"])

['hank had to wrap a lot of gifts for his family .',
 'he ran out of wrapping paper with 4 gifts to go .',
 'he went to the kitchen and found shopping bags .',
 'he cut up the bags to make sheets of paper .',
 'he used the paper to wrap the last of the gifts .']

In [50]:
from kogito.core.knowledge import KnowledgeGraph, Knowledge
from tqdm import tqdm

labels = roc_test.label.to_list()

for roc_result in tqdm(roc_results, total=len(roc_results)):
    context = text_to_list(roc_result["context"])
    graph = KnowledgeGraph([Knowledge(head=roc_result["head"], relation=roc_result["relation"], tails=roc_result["generated_tails"][0])])
    probs = linker.link(context=context, input_graph=graph)
    roc_result["prediction"] = 1 if probs[0][0] >= 0.5 else 0

100%|██████████| 100/100 [01:06<00:00,  1.50it/s]


In [51]:
import json

with open("data/roc_results.json", "w") as f:
    json.dump(roc_results, f, indent=2)

In [54]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import json

with open("data/roc_results.json") as f:
    roc_results = json.load(f)

labels = [roc_result["label"] for roc_result in roc_results]
predictions = [roc_result["prediction"] for roc_result in roc_results]

print(f"Accuracy={accuracy_score(labels, predictions)}, F1={f1_score(labels, predictions)}, Recall={recall_score(labels, predictions)}, Precision={precision_score(labels, predictions)}")

Accuracy=0.81, F1=0.8080808080808082, Recall=0.8, Precision=0.8163265306122449


### Context as a list

In [14]:
from kogito.core.knowledge import KnowledgeGraph, Knowledge
from tqdm import tqdm

labels = roc_test.label.to_list()
predictions = []

for _, row in tqdm(roc_test.iterrows(), total=len(roc_test)):
    context = text_to_list(row["context"])
    graph = KnowledgeGraph([Knowledge(head=row["head"], relation=row["relation"], tails=row["tail"])])
    probs = linker.link(context=context, input_graph=graph)
    predictions.append(1 if probs[0] >= 0.5 else 0)

100%|██████████| 10854/10854 [06:35<00:00, 27.42it/s]


In [15]:
len(predictions), len(labels)

(10854, 10854)

In [16]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

print(f"Accuracy={accuracy_score(labels, predictions)}, F1={f1_score(labels, predictions)}, Recall={recall_score(labels, predictions)}, Precision={precision_score(labels, predictions)}")

Accuracy=0.8941404090657822, F1=0.7769365171811298, Recall=0.7968936678614098, Precision=0.7579545454545454


### Context as a text

In [17]:
from kogito.core.knowledge import KnowledgeGraph, Knowledge
from tqdm import tqdm

labels = roc_test.label.to_list()
predictions = []

for _, row in tqdm(roc_test.iterrows(), total=len(roc_test)):
    context = " ".join(text_to_list(row["context"]))
    graph = KnowledgeGraph([Knowledge(head=row["head"], relation=row["relation"], tails=row["tail"])])
    probs = linker.link(context=context, input_graph=graph)
    predictions.append(1 if probs[0][0] >= 0.5 else 0)

  0%|          | 0/10854 [00:00<?, ?it/s]/root/.conda/envs/kogito/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.conda/envs/kogito/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
/root/.conda/envs/kogito/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:852: UserWarning: To copy construct from a tensor, it is recommended to use sourceTen

In [18]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

print(f"Accuracy={accuracy_score(labels, predictions)}, F1={f1_score(labels, predictions)}, Recall={recall_score(labels, predictions)}, Precision={precision_score(labels, predictions)}")

Accuracy=0.8944168048645661, F1=0.7775621118012422, Recall=0.7976901632815612, Precision=0.7584248390761076
